<a href="https://colab.research.google.com/github/swati-git/GoogleCollabNotebooks/blob/main/Building_a_RAG_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables


In [26]:
!pip install -q cohere==5.13.12 tiktoken==0.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 4.8 MB/s eta 0:00:00


In [24]:
import os
from google.colab import userdata

# Set the "COHERE_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["COHERE_API_KEY"] = userdata.get('cohere_api_key')

# Load Dataset


## Download Dataset (JSON)


The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model.


In [2]:
#!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles-with_embeddings.csv

--2025-02-25 10:31:37--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173646 (170K) [text/plain]
Saving to: ‘mini-llama-articles.csv’

mini-llama-articles 100%[===================>] 169.58K  --.-KB/s    in 0.1s    

2025-02-25 10:31:38 (1.32 MB/s) - ‘mini-llama-articles.csv’ saved [173646/173646]



# Generate Embedding


https://platform.openai.com/docs/guides/embeddings?lang=python

In [64]:
import cohere
import numpy as np
import os

co = cohere.Client(
    api_key=os.environ.get("COHERE_API_KEY")
)

# Defining a function that converts a text to embedding vector using OpenAI's Ada model.
def get_embedding(text):
    try:
        # Remove newlines
        text = text.replace("\n", " ")
        res = co.embed(
            texts=[text],
            model="embed-english-v3.0",
            input_type="search_document",
            embedding_types=["float"],
        )

        return res.embeddings.float

    except Exception as e:
        print(f"Error in get_embedding: {e}")  # Print the actual error message
        return None

if embedding is None:
    print(f"Embedding generation failed for text: '{text_to_embed}'")
else:
    print("Embedding generated successfully:")
    print(embedding)

Embedding generated successfully:
[[-0.028778076, -0.023544312, -0.059326172, -0.058654785, -0.035217285, -0.029830933, -0.033416748, 0.05456543, -0.014167786, 0.039001465, -0.03543091, -0.0010938644, 0.01977539, -0.031829834, 0.042510986, -0.010536194, 0.051971436, 6.252527e-05, 0.018585205, -0.006767273, -0.014968872, 0.039031982, -0.006061554, 0.019134521, 0.041656494, -0.031829834, 0.009468079, -0.013671875, -0.025497437, -0.04864502, 0.012046814, 0.044525146, 0.022399902, 0.025848389, 0.0050201416, 0.07531738, -0.018981934, 0.01826477, 0.014503479, -0.01272583, 0.018981934, -0.014564514, 0.013374329, -0.020874023, -0.034576416, 0.00932312, -0.0098724365, -0.0104904175, -0.0016479492, -0.04711914, 0.014930725, -0.031951904, 0.020339966, 0.015686035, -0.005821228, 0.012023926, -0.06097412, -0.02960205, 0.027359009, 0.03149414, 0.0057678223, -0.0043945312, 0.021987915, 0.000705719, 0.026031494, -0.041900635, 0.00020325184, 0.006000519, 0.027282715, 0.029586792, -0.0015506744, 0.00607

In [65]:
# Test the function
text_to_embed = "How many parameters LLaMA2 model has?"
#"Hello World"
embedding = get_embedding(text_to_embed)
print(embedding)

[[-0.051940918, 0.016220093, -0.026977539, 0.023254395, -0.03265381, 0.0013895035, -0.059295654, -0.034088135, -0.0793457, 0.018936157, 0.08276367, -0.009437561, -0.025558472, -0.076660156, 0.061950684, -0.005760193, 0.020645142, 0.029251099, -0.014053345, -0.015853882, -0.026535034, 0.029251099, -0.073913574, 0.097473145, 0.031585693, 0.010421753, -0.08892822, 0.040863037, 1.0788441e-05, 0.021209717, 0.008575439, -0.010910034, 0.020858765, 0.029556274, -0.022888184, 0.051086426, -0.018692017, 0.031188965, 0.023742676, -0.009811401, 0.04006958, -0.031433105, 0.04510498, 0.00121212, -0.00060653687, -0.019332886, 0.05722046, -0.0134887695, -0.040740967, -0.049560547, 0.00013530254, -0.0015735626, 0.018463135, -0.0020122528, 0.013435364, 0.008102417, 0.0151901245, -0.006816864, 0.019241333, -0.00642395, -0.03918457, 0.0211792, 0.032043457, -0.013908386, 0.015731812, -0.017593384, -0.043548584, 0.0345459, 0.025482178, 0.017425537, -0.0032138824, -0.03375244, 0.0129776, -0.03265381, 0.01543

In [15]:
import pandas as pd

In [46]:
from tqdm.notebook import tqdm
import numpy as np

# Generate embedding
if not load_embedding:
    print("Generating embeddings...")
    embeddings = []
    for index, row in tqdm(df.iterrows()):
        # df.at[index, 'embedding'] = get_embedding( row['chunk'] )
        embeddings.append(get_embedding(row["chunk"]))

    embeddings_values = pd.Series(embeddings)
    df.insert(loc=1, column="embedding", value=embeddings_values)

# Or, load the embedding from the file.
else:
    print("Loaded the embedding file.")
    # Load the file as a CSV
    df = pd.read_csv("mini-llama-articles-with_embeddings.csv")
    # Convert embedding column to an array
    df["embedding"] = df["embedding"].apply(lambda x: np.array(eval(x)), 0)

Loaded the embedding file.


<ipython-input-46-3ecc723900c7>:21: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df["embedding"] = df["embedding"].apply(lambda x: np.array(eval(x)), 0)


In [19]:
df

,Unnamed: 0,chunk,embedding
0,0,LLM Variants and Meta's Open Source Before she...,"[-0.01324542611837387, 0.005806310568004847, -..."
1,1,ational code model;Codel Llama - Python specia...,"[-0.02734695002436638, 0.01590133272111416, -0..."
2,2,"erm ""multimodal"" refers to their ability to pr...","[-0.04137444868683815, -0.0026930728927254677,..."
3,3,"es it matter? LLM connections, like the LlamaI...","[-0.03091380186378956, 0.0005604170146398246, ..."
4,4,understand data in the AI-driven future. Fro...,"[-0.00428031524643302, -0.0020621130242943764,..."
...,...,...,...
169,169,versity. In-breadth Evolving solves this probl...,"[-0.03063206933438778, 0.006754143629223108, -..."
170,170,"ns are done, the initial instruction dataset (...","[-0.017750171944499016, -0.0018687269184738398..."
171,171,"er, the Prompt should be as follows: Best Use...","[-0.019411379471421242, -0.006221776362508535,..."
172,172,"sis, and visualization.Machine Learning Pipeli...","[-0.004704236518591642, -0.011992035433650017,..."


# User Question


In [66]:
# Define the user question, and convert it to embedding.
QUESTION = "How many parameters LLaMA2 model has?"
QUESTION_emb = get_embedding(QUESTION)
print(QUESTION_emb)

[[-0.051940918, 0.016220093, -0.026977539, 0.023254395, -0.03265381, 0.0013895035, -0.059295654, -0.034088135, -0.0793457, 0.018936157, 0.08276367, -0.009437561, -0.025558472, -0.076660156, 0.061950684, -0.005760193, 0.020645142, 0.029251099, -0.014053345, -0.015853882, -0.026535034, 0.029251099, -0.073913574, 0.097473145, 0.031585693, 0.010421753, -0.08892822, 0.040863037, 1.0788441e-05, 0.021209717, 0.008575439, -0.010910034, 0.020858765, 0.029556274, -0.022888184, 0.051086426, -0.018692017, 0.031188965, 0.023742676, -0.009811401, 0.04006958, -0.031433105, 0.04510498, 0.00121212, -0.00060653687, -0.019332886, 0.05722046, -0.0134887695, -0.040740967, -0.049560547, 0.00013530254, -0.0015735626, 0.018463135, -0.0020122528, 0.013435364, 0.008102417, 0.0151901245, -0.006816864, 0.019241333, -0.00642395, -0.03918457, 0.0211792, 0.032043457, -0.013908386, 0.015731812, -0.017593384, -0.043548584, 0.0345459, 0.025482178, 0.017425537, -0.0032138824, -0.03375244, 0.0129776, -0.03265381, 0.01543

# Test Cosine Similarity


Calculating the similarity of embedding representations can help us to find pieces of text that are close to each other. In the following sample you see how the Cosine Similarity metric can identify which sentence could be a possible answer for the given user question. Obviously, the unrelated answer will score lower.


In [67]:
BAD_SOURCE_emb = get_embedding("The sky is blue.")
GOOD_SOURCE_emb = get_embedding("LLaMA2 model has a total of 2B parameters.")

In [71]:
!pip install scikit-learn

In [74]:
from sklearn.metrics.pairwise import cosine_similarity

# A sample that how a good piece of text can achieve high similarity score compared
# to a completely unrelated text.
print("> Bad Response Score:", cosine_similarity(QUESTION_emb, BAD_SOURCE_emb))
print("> Good Response Score:", cosine_similarity(QUESTION_emb, GOOD_SOURCE_emb))

> Bad Response Score: [[0.20671175]]
> Good Response Score: [[0.79657078]]


# Calculate Cosine Similarities


In [84]:
# The similarity between the questions and each part of the essay.
#cosine_similarities = cosine_similarity([QUESTION_emb], df["embedding"].tolist())

# The similarity between the questions and each part of the essay.
# Reshape the embedding column before calculating similarity
# cosine_similarities = cosine_similarity(
#     [QUESTION_emb[0]],  # Reshape QUESTION_emb to (1024,)
#     [x[0] for x in df["embedding"].tolist()]  # Extract embeddings and reshape to (1024,)
# )

# The similarity between the questions and each part of the essay.
# Reshape the embedding column before calculating similarity
# cosine_similarities = cosine_similarity(
#     QUESTION_emb,  # Use QUESTION_emb directly as it's already a 2D array
#     np.stack([np.array(x, dtype=np.float32) for x in df['embedding'].to_numpy()])  # Convert embeddings to numpy array with correct dtype and stack
# )


# Since QUESTION_emb is 1024 dimensions and df['embedding'] is 1536
# Reduce QUESTION_emb to 174 dimensions
# pca = PCA(n_components=174)
# reduced_question_emb = pca.fit_transform(QUESTION_emb)

# # Then reduce df['embedding'] to the same 174 dimensions
# embeddings_array = np.stack([np.array(x) for x in df['embedding']])
# reduced_df_emb = pca.transform(embeddings_array)  # Use same PCA model

# # Now compare
# cosine_similarities = cosine_similarity(reduced_question_emb, reduced_df_emb)

print("QUESTION_emb shape:", QUESTION_emb.shape)
print("embeddings_array shape:", embeddings_array.shape)

QUESTION_emb shape: (1, 1024)
embeddings_array shape: (174, 1536)


In [85]:
# Option 1: Skip dimensionality reduction entirely and use a different approach
from sklearn.metrics.pairwise import cosine_similarity

# Reshape single embedding to match format if needed
if len(QUESTION_emb.shape) == 1:
    QUESTION_emb = QUESTION_emb.reshape(1, -1)

# Truncate both embeddings to the smaller size
min_dim = min(QUESTION_emb.shape[1], embeddings_array.shape[1])
QUESTION_emb_truncated = QUESTION_emb[:, :min_dim]
embeddings_array_truncated = np.array([emb[:min_dim] for emb in embeddings_array])

# Calculate similarity with truncated vectors
cosine_similarities = cosine_similarity(QUESTION_emb_truncated, embeddings_array_truncated)

In [86]:
import numpy as np

number_of_chunks_to_retrieve = 3

# Sort the scores
highest_index = np.argmax(cosine_similarities)

# Pick the N highest scored chunks
indices = np.argsort(cosine_similarities[0])[::-1][:number_of_chunks_to_retrieve]
print(indices)

[93 94 89]


In [87]:
# Look at the highest scored retrieved pieces of text
for idx, item in enumerate(df.chunk[indices]):
    print(f"> Chunk {idx+1}")
    print(item)
    print("----")

> Chunk 1
ce and makes Llama 2 a more reliable language model for interactive applications. In the example below, on the left, it forgets to use an emoji after a few conversations. On the right, with Ghost Attention, even after having many conversations, it will remember the context and continue to use emojis in its response.  VI. Temporal Capability: A Leap in Information Organization Meta reported a groundbreaking temporal capability, where the model organizes information based on time relevance. Each question posed to the model is associated with a date, and it responds accordingly by considering the event date before which the question becomes irrelevant. For example, if you ask the question, "How long ago did Barack Obama become president?", its only relevant after 2008. This temporal awareness allows Llama 2 to deliver more contextually accurate responses, enriching the user experience further.  VII. Open Questions and Future Outlook Meta's open-sourcing of Llama 2 represents a s

# Augment the Prompt


In [89]:
os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')

In [90]:
import google.generativeai as genai
# Use the Gemini API to answer the questions based on the retrieved pieces of text.
try:
    # Formulating the system prompt and condition the model to answer only AI-related questions.
    system_prompt = (
        "You are an assistant and expert in answering questions from a chunks of content. "
        "Only answer AI-related question, else say that you cannot answer this question."
    )

    # Create a user prompt with the user's question
    prompt = (
        "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
        "Please provide an informative and accurate answer to the following question based on the avaiable context. Be concise and take your time. \nQuestion: {}\nAnswer:"
    )
    # Add the retrieved pieces of text to the prompt.
    prompt = prompt.format("".join(df.chunk[indices]), QUESTION)

    model = genai.GenerativeModel(model_name= "gemini-1.5-flash", system_instruction=system_prompt)

    result = model.generate_content(prompt,request_options={"timeout": 1000},)
    res = result.text

except Exception as e:
    print(f"An error occurred: {e}")

In [91]:
print(res)

LLaMA 2 is available in four sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.



## How Augmenting the Prompt can address knowledge cutoff limitations and hallucinations

In [ ]:
# Consider this as a retrieved chunk
# https://ai.meta.com/blog/meta-llama-3-1/
Example_chunk = """
Introducing Llama 3.1 Llama 3.1 405B is the first openly available model that rivals the top AI models when it comes to state-of-the-art capabilities in general knowledge, steerability, math, tool use, and multilingual translation. With the release of the 405B model, we’re poised to supercharge innovation—with unprecedented opportunities for growth and exploration. We believe the latest generation of Llama will ignite new applications and modeling paradigms, including synthetic data generation to enable the improvement and training of smaller models, as well as model distillation—a capability that has never been achieved at this scale in open source.
As part of this latest release, we’re introducing upgraded versions of the 8B and 70B models. These are multilingual and have a significantly longer context length of 128K, state-of-the-art tool use, and overall stronger reasoning capabilities. This enables our latest models to support advanced use cases, such as long-form text summarization, multilingual conversational agents, and coding assistants. We’ve also made changes to our license, allowing developers to use the outputs from Llama models—including the 405B—to improve other models. True to our commitment to open source, starting today, we’re making these models available to the community for download on llama.meta.com and Hugging Face and available for immediate development on our broad ecosystem of partner platforms. Model evaluations
For this release, we evaluated performance on over 150 benchmark datasets that span a wide range of languages. In addition, we performed extensive human evaluations that compare Llama 3.1 with competing models in real-world scenarios. Our experimental evaluation suggests that our flagship model is competitive with leading foundation models across a range of tasks, including GPT-4, GPT-4o, and Claude 3.5 Sonnet. Additionally, our smaller models are competitive with closed and open models that have a similar number of parameters.
Model Architecture As our largest model yet, training Llama 3.1 405B on over 15 trillion tokens was a major challenge. To enable training runs at this scale and achieve the results we have in a reasonable amount of time, we significantly optimized our full training stack and pushed our model training to over 16 thousand H100 GPUs, making the 405B the first Llama model trained at this scale.
To address this, we made design choices that focus on keeping the model development process scalable and straightforward. We opted for a standard decoder-only transformer model architecture with minor adaptations rather than a mixture-of-experts model to maximize training stability.
We adopted an iterative post-training procedure, where each round uses supervised fine-tuning and direct preference optimization. This enabled us to create the highest quality synthetic data for each round and improve each capability’s performance.
Compared to previous versions of Llama, we improved both the quantity and quality of the data we use for pre- and post-training. These improvements include the development of more careful pre-processing and curation pipelines for pre-training data, the development of more rigorous quality assurance, and filtering approaches for post-training data.
"""

In [ ]:
QUESTION = "How many parameters LLaMA 3.1 model has?"

# Formulating the system prompt
system_prompt = (
        "You are an assistant and expert in answering questions from a chunks of content. "
        "Only answer AI-related question, else say that you cannot answer this question."
    )
# Combining the system prompt with the user's question
prompt = (
        "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
        "Please provide an informative and accurate answer to the following question based on the avaiable context. Be concise and take your time. \nQuestion: {}\nAnswer:"
    )
prompt = prompt.format(Example_chunk, QUESTION)

model = genai.GenerativeModel(model_name= "gemini-1.5-flash", system_instruction=system_prompt)

#Gemini API call
result = model.generate_content(prompt,request_options={"timeout": 1000},)
res = result.text
print(res)

The LLaMA 3.1 model has **405 billion parameters**. 



# Without Augmentation


Test the Gemini API to answer the same question without the addition of retrieved documents. Basically, the LLM will use its knowledge to answer the question.


In [ ]:
QUESTION = "How many parameters LLaMA 3 model has?"

# Formulating the system prompt
system_prompt = "You are an assistant and expert in answering questions."

# Combining the system prompt with the user's question
prompt = "Be concise and take your time to answer the following question. \nQuestion: {}\nAnswer:"
prompt = prompt.format(QUESTION)

model = genai.GenerativeModel(model_name= "gemini-1.5-flash", system_instruction=system_prompt)

#Gemini API call
result = model.generate_content(prompt,request_options={"timeout": 1000},)
res = result.text

In [ ]:
print(res)

LLaMA 3 has **34 billion parameters**. 

